In [1]:
import numpy as np
import pandas as pd

In [2]:
class multiclass():
    def __init__(self,input = 3, capas=1,nodos=1,clases = 3):
        self.capas = capas
        self.nodos = nodos
        self.forma = (self.capas,self.nodos)
        self.clases = clases
        self.input = input
    def inicializar(self):
        self.red = []
        w = np.random.normal(size = (self.input,self.nodos))
        b = np.random.normal(size = self.nodos)
        self.red.append([w,b])
        for capa in range(self.capas):
            w = np.random.normal(size = (self.nodos,self.nodos))
            b = np.random.normal(size = self.nodos)
            self.red.append([w,b])
        w_f = np.random.normal(size = [self.nodos,self.clases])
        b_f = np.random.normal(size = self.clases)
        self.red.append([w_f,b_f])
    def mutar(self,alfa=0.1):
        red = []
        for r in self.red:
            r0 = r[0]+alfa*np.random.normal(size = r[0].shape)
            r1 = r[1]+alfa*np.random.normal(size = r[1].shape)
            red.append([r0,r1])
        hijo = multiclass(self.input,self.capas,self.nodos,self.clases)
        hijo.inicializar()
        hijo.red = red
        return hijo
    def cruzar(self,other):
        if self.forma == other.forma:
            hijo = multiclass(self.input,self.capas,self.nodos,self.clases)
            hijo.inicializar()
            red = []
            for i in range(len(self.red)):
                r0 = (self.red[i][0] + other.red[i][0])/2.0
                r1 = (self.red[i][1] + other.red[i][1])/2.0
                red.append([r0,r1])
            hijo.red = red
            return hijo
        else:
            return None
    def sigmoid(self,x):
        return 1.0/(1.0 + np.exp(-x))
    def softmax(self,x):
        x = np.exp(x)/(np.exp(x).sum())
        return x
    def predecir(self,entrada):
        for rr in range(len(self.red)):
            r = self.red[rr]
            entrada = np.dot(entrada,r[0])
            entrada = entrada + r[1]
            entrada = self.sigmoid(entrada)
        entrada = self.softmax(entrada)
        entrada = entrada.argmax()
        return entrada
    def evaluar(self,entrada,resultado):
        entrada = self.predecir(entrada)
        if float(entrada) == float(resultado):
            return 1
        return 0

        
            
    

In [3]:
def crear_poblacion(poblacion_por_especie,especies,input,nodos,clases):
    poblacion = []
    for i in range(poblacion_por_especie):
        for j in especies:
            individuo = multiclass(input = input,capas = j,nodos = nodos,clases = clases)
            individuo.inicializar()
            poblacion.append(individuo)
    return poblacion

In [4]:
def evaluar_poblacion(poblacion,muestra):
    scores = []
    for p in poblacion:
        score = 0
        count = 0
        for r in muestra.iterrows():
            v = r[1].target
            u = r[1]
            del u['target']
            score += p.evaluar(u.values,v)
            count += 1
        score = float(score / float(count))
        scores.append(score)
    return scores

In [5]:
def predecir(best_model,test):
    scores = []
    for r in test.iterrows():
        u = r[1]
        score = p.predecir(u)
        scores.append(score)
    return scores

In [6]:
titanic = pd.read_csv('mnist/train.csv')
titanic = titanic.rename(columns={'label':'target'})


In [7]:
titanic.target.head()

0    1
1    0
2    1
3    4
4    0
Name: target, dtype: int64

In [8]:
titanic_train = titanic.sample(38000)
titanic_test = titanic.loc[~titanic.index.isin(titanic_train.index),:]



In [9]:
titanic_train.target.head()

16211    3
40694    4
24764    2
10119    4
27711    0
Name: target, dtype: int64

In [10]:
poblacion = crear_poblacion(1000,[1],784,64,10)
poblacion = pd.DataFrame(list(zip(poblacion,range(len(poblacion)))))
poblacion.columns = ['model','score']

In [ ]:
alfa = 1
n=10
for i in range(100000):
    scores = evaluar_poblacion(poblacion.model.tolist(),titanic_train.sample(n))
    poblacion.score = scores
    poblacion = poblacion.sort_values('score',ascending=False)[0:int(len(poblacion)/2)]
    nueva_poblacion = []
    alfa = 1
    for p in poblacion.model.tolist():
        hijo = p.mutar(alfa = alfa)
        nueva_poblacion.append(hijo)
    nueva_poblacion = pd.DataFrame(list(zip(nueva_poblacion,range(len(nueva_poblacion)))))
    nueva_poblacion.columns = ['model','score']
    poblacion = poblacion.append(nueva_poblacion).reset_index(drop=True)
    if i%100==0:
        scores = evaluar_poblacion(poblacion.model.tolist(),titanic_test.sample(1000))
        poblacion.score = scores
        print(poblacion.sort_values('score',ascending=False).head())


/home/jesus/anaconda3/envs/tensorflow/lib/python3.4/site-packages/ipykernel_launcher.py:44: RuntimeWarning: overflow encountered in exp


                                              model  score
213  <__main__.multiclass object at 0x7f57734de5f8>  0.395
803  <__main__.multiclass object at 0x7f57737e5898>  0.391
866  <__main__.multiclass object at 0x7f5763022588>  0.389
449  <__main__.multiclass object at 0x7f5773519d30>  0.389
447  <__main__.multiclass object at 0x7f5773571f98>  0.389
                                              model  score
106  <__main__.multiclass object at 0x7f5762f50128>  0.365
810  <__main__.multiclass object at 0x7f5762f14e80>  0.360
722  <__main__.multiclass object at 0x7f5762f73048>  0.358
843  <__main__.multiclass object at 0x7f57734e96a0>  0.358
119  <__main__.multiclass object at 0x7f57736f6748>  0.358


In [ ]:
scores = evaluar_poblacion(poblacion.model.tolist(),titanic_test)
poblacion.score = scores
poblacion.sort_values('score',ascending=False)

In [ ]:
best_model = poblacion.sort_values('score',ascending = False).iloc[0,0]

In [ ]:
titanic = pd.read_csv('mnist/test.csv')


In [ ]:
titanic.head()

In [ ]:
resultado = pd.read_csv('mnist/test.csv')
scores = predecir(best_model,titanic)
resultado['Label'] = scores
resultado.Label = resultado.Label.round().astype(int)
resultado['ImageId']=list(range(1,len(resultado)+1))

resultado[['ImageId','Label']].to_csv('mnist/result.csv',index=False)